## Setup

In [1]:
%matplotlib inline

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

# External modules
import ast
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [2]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[0])
os.chdir(main_dir)
print('main dir:',main_dir)

main dir: /Users/pablo/OneDrive/data-science/github/isuelogit


In [3]:
# Internal modules
from src import isuelogit as isl

In [4]:
# Seed
_SEED = 2022

## Read network data from Fresno

In [5]:
network_name = 'Fresno'

# To write estimation report and set seed for all algorithms involving some randomness  
estimation_reporter = isl.writer.Reporter(
    folderpath = isl.config.dirs['output_folder'] + 'estimations/' + network_name, seed = _SEED)

# Reader of geospatial and spatio-temporal data
data_reader = isl.etl.DataReader(network_key=network_name)

# First Tuesday of October, 2019
data_reader.select_period(date='2019-10-01', hour=16)

# Read nodes data
nodes_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/nodes/'  + 'fresno-nodes-data.csv')

# Read spatiotemporal link data
links_df = pd.read_csv(
    isl.dirs['input_folder'] + 'network-data/links/' + str(data_reader.options['selected_date'])+ '-fresno-link-data.csv',
    converters={"link_key": ast.literal_eval,"pems_id": ast.literal_eval})


Selected date is 2019-10-01, Tuesday at 16:00


## Build network

In [6]:
network_generator = isl.factory.NetworkGenerator()

A = network_generator.generate_adjacency_matrix(links_keys=list(links_df['link_key'].values))

fresno_network = \
    network_generator.build_fresno_network(A=A, links_df=links_df, nodes_df=nodes_df, network_name= network_name)


Creating Fresno network

Nodes: 1789, Links: 2413


### OD matrix

In [7]:
network_generator.read_OD(network=fresno_network, sparse=True)

Matrix Q (1789, 1789) read in 0.0[s] with sparse format
66266.34839999994 trips were loaded among 6970 o-d pairs


### Paths

In [8]:
paths_generator = isl.factory.PathsGenerator()

# # Generate and Load paths in network
# paths_generator.load_k_shortest_paths(network = fresno_network, k=3)

paths_generator.read_paths(network=fresno_network, update_incidence_matrices=True)

20840 paths were read in 7.3[s]| 100.0% 
20840 paths were loaded in the network
Updating incidence matrices
Matrix D (2413, 20840) generated in 25.8[s]
Matrix M (6970, 20840) generated in 1.8[s] 
Matrix C (20840, 20840) generated in 5.7[s]


### Link performance functions

In [9]:
bpr_parameters_df = pd.DataFrame({'link_key': links_df['link_key'],
                                  'alpha': links_df['alpha'],
                                  'beta': links_df['beta'],
                                  'tf': links_df['tf'],
                                  'k': pd.to_numeric(links_df['k'], errors='coerce', downcast='float')
                                  })

# Normalize free flow travel time between 0 and 1
bpr_parameters_df['tf'] = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(np.array(bpr_parameters_df['tf']).reshape(-1, 1)))

fresno_network.set_bpr_functions(bprdata=bpr_parameters_df)

### Feature Engineering

In [10]:
fresno_network.load_features_data(links_df, link_key = 'link_key')

isl.etl.feature_engineering_fresno(links=fresno_network.links, network=fresno_network)

Features values of links with a type different than LWRLK were set to 0
New features: ['low_inc', 'high_inc', 'no_incidents', 'no_bus_stops', 'no_intersections', 'tt_sd_adj', 'tt_reliability']


In [11]:
features_list = ['median_inc', 'intersections', 'incidents', 'bus_stops', 'median_age',
                 'tt_avg', 'tt_sd','tt_var', 'tt_cv', 'tt_sd_adj','tt_reliability',
                 'speed_ref_avg', 'speed_avg', 'speed_hist_avg','speed_sd','speed_hist_sd','speed_cv']


# Normalization of features to range [0,1]
linkdata = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(fresno_network.Z_data[features_list].values))
linkdata.columns = features_list
linkdata.insert(0, 'link_key', fresno_network.links_keys)

fresno_network.load_features_data(linkdata)

### Traffic counts

In [12]:
counts_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/links/' \
                            + str(data_reader.options['selected_date']) + '-fresno-link-counts' + '.csv',
                        converters={'link_key': ast.literal_eval})

counts = dict(zip(counts_df['link_key'].values, counts_df['counts'].values))

fresno_network.load_traffic_counts(counts=counts)

## Descriptive statistics

### Link coverage

In [13]:
total_counts_observations = np.count_nonzero(~np.isnan(np.array(list(counts.values()))))

total_links = np.array(list(counts.values())).shape[0]

print('\nTotal link counts observations: ' + str(total_counts_observations))
print('Link coverage: ' + "{:.1%}".format(round(total_counts_observations / total_links, 4)))


Total link counts observations: 141
Link coverage: 5.8%


### Networks topology

In [14]:
isl.descriptive_statistics.summary_table_networks([fresno_network])

,network,nodes,links,ods,paths
0,Fresno,1789,2413,6970,20840


### Links data

In [15]:
summary_table_links_df = fresno_network.Z_data
estimation_reporter.write_table(df = summary_table_links_df, filename = 'links_data.csv', float_format = '%.3f')
summary_table_links_df

,link_type,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],...,median_age,incidents,bus_stops,intersections,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_reliability
0,LWRLK,0.15,4.0,0.098,1.800000e+03,0,NaN,1800.0,0.098,45,...,0.664530,0.000000,0.0,0.285714,1,0,1,1,0,0.298684
1,LWRLK,0.15,4.0,0.169,1.800000e+03,0,NaN,1800.0,0.169,50,...,0.735043,0.142857,0.0,0.000000,1,0,0,1,1,0.328970
2,LWRLK,0.15,4.0,0.396,2.400000e+03,0,NaN,2400.0,0.396,65,...,0.735043,0.285714,0.0,0.000000,1,0,0,1,1,0.000000
3,LWRLK,0.15,4.0,0.192,2.000000e+03,0,NaN,2000.0,0.192,25,...,0.735043,0.000000,0.0,0.285714,1,0,1,1,0,0.334352
4,LWRLK,0.15,4.0,0.105,1.800000e+03,0,NaN,1800.0,0.105,35,...,0.735043,0.000000,0.0,0.142857,1,0,1,1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.000000,0.000000,0.0,0.000000,0,0,0,0,0,0.000000
2409,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.000000,0.000000,0.0,0.000000,0,0,0,0,0,0.000000
2410,LWRLK,0.15,4.0,0.045,1.800000e+03,0,NaN,1800.0,0.045,65,...,0.617521,0.071429,0.0,0.285714,1,0,0,1,0,0.380265
2411,LWRLK,0.15,4.0,0.107,2.400000e+03,0,NaN,2400.0,0.107,65,...,0.961538,0.285714,0.5,0.428571,0,1,0,0,0,0.387156


In [16]:
summary_table_links_df.describe()

,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],inrix_id,...,median_age,incidents,bus_stops,intersections,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_reliability
count,2.413000e+03,2413.0,2413.000000,2.413000e+03,2413.000000,141.000000,2413.0,2413.000000,2413.000000,1.468000e+03,...,2413.000000,2413.000000,2413.00000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000
mean,1.500000e-01,4.0,0.150823,1.439305e+09,0.058433,1126.496454,inf,0.150823,28872.664318,9.131292e+08,...,0.474693,0.038180,0.12400,0.167897,0.525073,0.186490,0.553668,0.517199,0.121840,0.192557
std,2.776133e-17,0.0,0.172458,2.261116e+09,0.234610,428.254089,NaN,0.172458,45293.922006,6.019207e+08,...,0.322585,0.132807,0.27655,0.202384,0.499474,0.389582,0.497214,0.499808,0.327169,0.180654
min,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,63.000000,1800.0,0.000000,15.000000,1.685461e+08,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,854.000000,1800.0,0.000000,40.000000,4.416712e+08,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.500000e-01,4.0,0.121000,1.800000e+03,0.000000,1073.000000,1800.0,0.121000,45.000000,4.498919e+08,...,0.594017,0.000000,0.00000,0.142857,1.000000,0.000000,1.000000,1.000000,0.000000,0.289861
75%,1.500000e-01,4.0,0.205000,4.990000e+09,0.000000,1343.000000,NaN,0.205000,99999.000000,1.626675e+09,...,0.707265,0.000000,0.00000,0.285714,1.000000,0.000000,1.000000,1.000000,0.000000,0.358503
max,1.500000e-01,4.0,2.113000,4.990000e+09,1.000000,2354.000000,inf,2.113000,99999.000000,1.626774e+09,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Bilevel Estimation

In [21]:
utility_function = isl.estimation.UtilityFunction(
    features_Y=['tt'],
    initial_values={'tt': 0},
)

equilibrator_norefined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    max_iters=100,
    method='fw',
    iters_fw=10,
#     column_generation={'n_paths': 4,
#                        'ods_coverage': 0.1,
#                        'paths_selection': 3},
    path_size_correction=1
)

outer_optimizer_norefined = isl.estimation.OuterOptimizer(
    method='ngd',
    iters=1,
    eta=1e-1,
)

learner_norefined = isl.estimation.Learner(
    equilibrator=equilibrator_norefined,
    outer_optimizer=outer_optimizer_norefined,
    utility_function=utility_function,
    network=fresno_network,
    name = 'norefined'
)

equilibrator_refined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    max_iters=100,
    method='fw',
    iters_fw=10,
    path_size_correction=1
)

outer_optimizer_refined = isl.estimation.OuterOptimizer(
    method='lm',
    # method='ngd',
    iters=1,
    # eta=5e-1,
)

learner_refined = isl.estimation.Learner(
    network=fresno_network,
    equilibrator=equilibrator_refined,
    outer_optimizer=outer_optimizer_refined,
    utility_function=utility_function,
    name = 'refined'
)

### a) Benchmark predictions

#### Naive prediction using mean counts

In [18]:
mean_counts_prediction_loss, mean_count_benchmark_model \
    = isl.estimation.mean_count_prediction(counts=np.array(list(counts.values()))[:, np.newaxis])

print('\nObjective function under mean count prediction: ' + '{:,}'.format(round(mean_counts_prediction_loss, 1)))


Objective function under mean count prediction: 101,141,667.7


#### Naive prediction where travelers make equilikely choices

In [19]:
equilikely_prediction_loss, predicted_counts_equilikely \
    = isl.estimation.loss_counts_equilikely_choices(
    network = fresno_network,
    equilibrator=equilibrator_norefined,
    counts=fresno_network.counts_vector,
    utility_function=utility_function)

print('Objective function under equilikely route choices: ' + '{:,}'.format(round(equilikely_prediction_loss, 1)))

Objective function under equilikely route choices: 272,997,589.4


### b) Model with travel time feature only

#### No refined stage

In [22]:
learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = False)

theta_norefined = learning_results_norefined[best_iter_norefined]['theta']


Bilevel optimization for Fresno network 

Iteration : 1/10

Initial theta: {'tt': '0.0E+00'}

SUE via fw (max iters: 100)
Path size correction with factor 1

Equilibrium gaps: ['0E+00']
Initial Fisk Objective: -179,604.51
Final Fisk Objective: -179,604.51
Improvement Fisk Objective: 0.00%
Final gap: 0E+00. Acc. bound: 1E-04. Time: 213.2 [s]
Initial objective: 273,253,642
Initial RMSE: 1392.1
Initial Normalized RMSE: 0.629

Iteration : 2/10

Estimating parameters via ngd (1 iters, eta = 1.0E-01)

theta: {'tt': '-1.0E-01'}------| 0.0% 
time: 188.9[s]

SUE via fw (max iters: 100)
Path size correction with factor 1

Equilibrium gaps: ['0E+00']
Initial Fisk Objective: -183,589.11
Final Fisk Objective: -183,071.23
Improvement Fisk Objective: 0.28%
Final gap: 0E+00. Acc. bound: 1E-04. Time: 94.6 [s]

Time current iteration: 283.8 [s]
Current objective_value: 252,261,228
Current objective improvement: 7.68%
Current RMSE: 1337.6
Current Normalized RMSE: 0.604
Marginal objective improvement: 7.

#### Remove traffic count observations of links where flows do not change over iterations 

In [23]:
# new_counts, removed_links_keys = isl.etl.get_informative_links_fresno(
#     learning_results=learning_results_norefined,
#     network=fresno_network)

# fresno_network.load_traffic_counts(new_counts)

#### Refined stage

In [ ]:
learner_refined.utility_function.initial_values = theta_norefined

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = False)

theta_refined = learning_results_refined[best_iter_refined]['theta']


Bilevel optimization for Fresno network 

Iteration : 1/10

Initial theta: {'tt': '-9.0E-01'}

SUE via fw (max iters: 100)
Path size correction with factor 1

Equilibrium gaps: ['3E-04', '3E-06']
Initial Fisk Objective: -211,803.06
Final Fisk Objective: -207,646.57
Improvement Fisk Objective: 1.96%
Final gap: 3E-06. Acc. bound: 1E-04. Time: 98.3 [s]
Initial objective: 226,335,022
Initial RMSE: 1267.0
Initial Normalized RMSE: 0.572

Iteration : 2/10

Estimating parameters via lm-rev (1 iters)

Damping factors: ['1.0E-02']---| 0.0% 
theta: {'tt': '-1.9E+00'}
time: 83.3[s]

SUE via fw (max iters: 100)
Path size correction with factor 1

Equilibrium gaps: ['3E-04', '0E+00']
Initial Fisk Objective: -245,481.84
Final Fisk Objective: -236,522.3
Improvement Fisk Objective: 3.65%
Final gap: 0E+00. Acc. bound: 1E-04. Time: 101.4 [s]

Time current iteration: 184.8 [s]
Current objective_value: 218,917,512
Current objective improvement: 3.28%
Current RMSE: 1246.0
Current Normalized RMSE: 0.563
Mar

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined,
    theta_refined= theta_refined,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

# Summary with most relevant options, prediction error, initial parameters, etc
estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function)

# Write tables with results on learning and inference
estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')

#### Visualizations

##### Convergence

In [ ]:
results_df = isl.descriptive_statistics \
    .get_loss_and_estimates_over_iterations(results_norefined=learning_results_norefined
                                            , results_refined=learning_results_refined)

fig = isl.visualization.Artist().convergence(
    results_norefined_df=results_df[results_df['stage'] == 'norefined'],
    results_refined_df=results_df[results_df['stage'] == 'refined'],
    filename='convergence_' + fresno_network.key,
    methods=[outer_optimizer_norefined.method.key, outer_optimizer_refined.method.key],
    folder = estimation_reporter.dirs['estimation_folder'],
    simulated_data = False
)

##### Distribution of errors across link counts

In [ ]:
best_predicted_counts_norefined = np.array(list(learning_results_norefined[best_iter_norefined]['x'].values()))[:, np.newaxis]
best_predicted_counts_refined = np.array(list(learning_results_refined[best_iter_refined]['x'].values()))[:, np.newaxis]

fig, axs = plt.subplots(1, 2, sharey='all', tight_layout=True, figsize=(8, 4))

axs[0].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_norefined))
axs[1].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_refined))

for axi in [axs[0], axs[1]]:
    axi.tick_params(axis='x', labelsize=16)
    axi.tick_params(axis='y', labelsize=16)

plt.show()

fig.savefig(estimation_reporter.dirs['estimation_folder'] + '/' + 'distribution_predicted_error_counts.pdf',
            pad_inches=0.1, bbox_inches="tight")

### c) Model with all features

In [ ]:
utility_function_full_model = isl.estimation.UtilityFunction(
    features_Y=['tt'],
    features_Z= ['tt_cv', 'incidents', 'intersections', 'bus_stops', 'median_inc'],
    fixed = {'tt':True},
    parameters_constraints = {'fixed':True},
    signs = {'tt': '-', 'tt_cv':'-', 'incidents':'-', 'intersections':'-', 'bus_stops':'-', 'median_inc':'+'}
)

# Update utility functions of no refined and refined learners
learner_norefined.utility_function = utility_function_full_model
learner_refined.utility_function = utility_function_full_model

#Initialize value with the estimate obtained from b)
learner_norefined.utility_function.initial_values = theta_refined

estimation_reporter = isl.writer.Reporter(
    folderpath = isl.config.dirs['output_folder'] + 'estimations/' + network_name, seed =_SEED)

#### No refined stage

In [ ]:
learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True)

theta_norefined_full_model = learning_results_norefined[best_iter_norefined]['theta']

#### Refined stage

In [ ]:
learner_refined.utility_function.initial_values = theta_norefined_full_model

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True)

theta_refined_full_model = learning_results_refined[best_iter_refined]['theta']

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_full_model,
    theta_refined= theta_refined_full_model,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_full_model)

estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function_full_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')

### d) Model including features with parameters with expected sign

In [ ]:
features_Y,features_Z = isl.estimation.feature_selection(utility_function_full_model,
                                                          theta = theta_refined_full_model,
                                                          criterion = 'sign')

utility_function_feature_selection_model = isl.estimation.UtilityFunction(
    features_Y = features_Y,
    features_Z = features_Z
)

learner_norefined.utility_function = utility_function_feature_selection_model
learner_refined.utility_function = utility_function_feature_selection_model

learner_norefined.utility_function.initial_values = theta_refined_full_model

estimation_reporter = isl.writer.Reporter(
    folderpath = isl.config.dirs['output_folder'] + 'estimations/' + network_name, seed = _SEED)

#### No refined stage

In [ ]:
learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, 
                                            link_report=False, iteration_report = True)

theta_norefined_feature_selection_model = learning_results_norefined[best_iter_norefined]['theta']

#### Refined stage

In [ ]:
learner_refined.utility_function.initial_values = theta_norefined_feature_selection_model

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, 
                                          link_report=False, iteration_report = True)

theta_refined_feature_selection_model = learning_results_refined[best_iter_refined]['theta']

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_feature_selection_model,
    theta_refined= theta_refined_feature_selection_model,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_feature_selection_model)

estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function_feature_selection_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')

### e) Model with all features and constrained optimization on expected signs of parameters

In [ ]:
utility_function_constrained_model = isl.estimation.UtilityFunction(
    features_Y=['tt'],
    features_Z= ['tt_cv'],
    signs = {'tt': '-', 'tt_cv':'-'}
    fixed = {'tt': True, 'tt_cv':False}
)

# Update utility functions of no refined and refined learners
learner_norefined.utility_function = utility_function_constrained_model
learner_refined.utility_function = utility_function_constrained_model

#Initialize value with the estimate obtained from b)
learner_norefined.utility_function.initial_values = theta_refined

#### No refined stage

In [ ]:
learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=5, alpha=0.05, link_report=False, iteration_report = True, 
                                            parameters_constraints = {'fixed': True, 'sign': True})

theta_norefined_full_model = learning_results_norefined[best_iter_norefined]['theta']

#### Refined stage